In [1]:
import Base.copy
using Distributions
include("Mondrian_Forest_Classifier.jl"); 

#### Outcommented stuff will be loaded from the files once bugs are fixed

In [2]:
# # just utils that should come from the file

# type Mondrian_Node
#     parent::Nullable{Mondrian_Node}
#     left::Nullable{Mondrian_Node}
#     right::Nullable{Mondrian_Node}
#     τ::Float64                      # split time
#     node_type::Array{Bool,1}        # node,leaf,root
#     δ::Nullable{Int}                # split dimension
#     ζ::Nullable{Float64}            # split position
#     Θ::Nullable{Axis_Aligned_Box}   # data boxes B
#     tab::Array{Int}                 # tables serving dish k Chinese restaurant process (CRP) / if number of instances of this class at the node is greater than 0 -> put 1, else put 0 -> binary information: is class k present at the node?
#     c::Array{Int}                   # customers eating dish k, tab[k] = min(c[k],1) IKN approx / number of instances of each class at the node 
#     Gₚ::Array{Float64}              # posterior mean (predictive probability)
#     indices::Array{Int}             # data within the boxes Θ
# end

# # some construction options

# function Mondrian_Node(τ, node_type, tab, c, Gₚ)  #constructor if we already know these things
#     N = Mondrian_Node(Nullable{Mondrian_Node}(),
#                       Nullable{Mondrian_Node}(),
#                       Nullable{Mondrian_Node}(),
#                       τ,node_type,
#                       Nullable{Int}(),
#                       Nullable{Float64}(),
#                       Nullable{Axis_Aligned_Box}(),
#                       tab,
#                       c,
#                       Gₚ,
#                       Array{Int}())
#     return N
# end

# function Mondrian_Node()   #constructor if we know nothing (do we need this one?)
#     N = Mondrian_Node(Nullable{Mondrian_Node}(),
#                       Nullable{Mondrian_Node}(),
#                       Nullable{Mondrian_Node}(),
#                       0,[true,false,false],
#                       Nullable{Int}(),
#                       Nullable{Float64}(),
#                       Nullable{Axis_Aligned_Box}(),
#                       Array{Int}(),
#                       Array{Int}(),
#                       Array{Float64}(),
#                       Array{Int}())
#     return N
# end

# function Mondrian_Node(τ::Float64) # constructor if we just know τ
#     N = Mondrian_Node(Nullable{Mondrian_Node}(),
#                       Nullable{Mondrian_Node}(),
#                       Nullable{Mondrian_Node}(),
#                       τ,[true,false,false],
#                       Nullable{Int}(),
#                       Nullable{Float64}(),
#                       Nullable{Axis_Aligned_Box}(),
#                       Array{Int}(),
#                       Array{Int}(),
#                       Array{Float64}(),
#                       Array{Int}(1))
#     return N
# end

# function Mondrian_Node(τ::Float64, node_type::Array{Bool,1})  #constructor if we only know τ and the node type
#     N = Mondrian_Node(Nullable{Mondrian_Node}(),
#                       Nullable{Mondrian_Node}(),
#                       Nullable{Mondrian_Node}(),
#                       τ,
#                       node_type,
#                       Nullable{Int}(),
#                       Nullable{Float64}(),
#                       Nullable{Axis_Aligned_Box}(),
#                       Array{Int}(),
#                       Array{Int}(),
#                       Array{Float64}(),
#                       Array{Int}())
#     return N
# end

# # only really need leaves + root directly
# type Mondrian_Tree  #type that stores the root, leaves and number of nodes
#     root::Nullable{Mondrian_Node}
#     leaves::Array{Mondrian_Node,1}
#     number_of_nodes::Int
# end

# function Mondrian_Tree()   #constructor for empty mondrian tree, zero leaves yet
#     T = Mondrian_Tree(Nullable{Mondrian_Node}(),Array{Mondrian_Node,1}(),0)
# end

# function Mondrian_Tree(ϵ::Mondrian_Node) #constructor for empty tree + root node
#     T = Mondrian_Tree(ϵ,Array{Mondrian_Node,1}(),0)
# end





# function Sample_Mondrian_Block!(j::Mondrian_Node,
#                                 Θ::Axis_Aligned_Box,
#                                 λ::Float64,
#                                 Tree::Mondrian_Tree,
#                                 X::Array{Float64,2},
#                                 Y::Array{Int64})
#     # sample the time
#     E = rand(Exponential(1/Linear_dimension(Θ)))
#     if j.node_type[3]==true
#         τₚ = 0
#     else
#         τₚ = (get(j.parent)).τ
#     end
#     # if split occured in time
#     if τₚ + E < λ
#         # get split dimension and cut position
#         # A2 -> lines 6,7
#         d,x = sample_split_dimension(Θ)
#         # update node j's data
#         j.δ = d
#         j.ζ = x
#         j.τ = τₚ+E
#         Θᴸ = copy(Θ)
#         # look at this copy
#         Θᴿ = copy(Θ)
#         # Left and Right children have constricted boxes
#         Θᴸ.Intervals[d,2]=x
#         Θᴿ.Intervals[d,1]=x
#         # check there is actually data here
#         Xᴿ = get_data_indices(Θᴿ,X,d)
#         Xᴸ = get_data_indices(Θᴸ,X,d)
#         # strictly binary tree
#         if (size(Xᴿ,1)>0 && size(Xᴸ,1)>0)
#             right = Mondrian_Node(0.0, [true,false,false])
#             right.parent = j
#             # data changes A2 -> lines 8,9,10
#             right.Θ = Θᴿ
#             get_count(right, Y[Xᴿ], length(j.c))
#             right.Gₚ=zeros(size(j.c,1))
#             j.right = right

#             left = Mondrian_Node(0.0, [true,false,false])
#             left.parent = j
#             left.Θ = Θᴸ
#             get_count(left, Y[Xᴸ], length(j.c))
#             left.Gₚ = zeros(size(j.c,1))
#             j.left = left

#             # recurse
#             Sample_Mondrian_Block!(left, get(left.Θ), λ, Tree, X[Xᴸ,:], Y[Xᴸ])
#             Sample_Mondrian_Block!(right,get(right.Θ),λ, Tree, X[Xᴿ,:], Y[Xᴿ])
#         # set j as a leaf for no data/ not binary
#         else
#             j.τ = λ
#             j.node_type = [false,true,false]
#             push!(Tree.leaves,j)
#             return
#         end
#     # set j as leaf for time out
#     else
#         j.τ = λ
#         j.node_type = [false,true,false]
#         push!(Tree.leaves,j)
#         return
#     end
# end


# # returns any data from D contained in the boxes of Θ
# # idea: feed the split dimension as input, only look at that entry in every instance instead of all of them
# function get_data_indices(Θ::Axis_Aligned_Box, X::Array{Float64,2}, dim::Int64)
#     # this function cause large memory allocation according
#     # to @time but the system does not record any
#     # large memory allocation -> ram does not get increased
#     # at all!
#     indices = []
#     for i in 1:size(X,1)
#         if !(X[i,dim] < Θ.Intervals[dim,1] || X[i,dim] > Θ.Intervals[dim,2])
#             push!(indices,i)
#         end
#     end
#     return indices
# end
# # returns any data from D contained in the boxes of Θ
# function get_data_indices(Θ::Axis_Aligned_Box, X::Array{Float64,2})
#     # this function cause large memory allocation according
#     # to @time but the system does not record any
#     # large memory allocation -> ram does not get increased
#     # at all!
#     indices = []
#     include = false
#     for i in 1:size(X,1)
#         for j in 1:size(Θ.Intervals,1)
#             if (X[i,j] < Θ.Intervals[j,1] || X[i,j] > Θ.Intervals[j,2])
#                 include = false
#                 break
#             end
#             include = true
#         end
#         if (include)
#             push!(indices, i)
#         end
#     end
#     return indices
# end

# # counts the proportion of each label in
# # the data contained within the leaf nodes
# # Θ
# function initialize_posterior_leaf_counts!(Tree, D)  #MT=tree=Tree, here D is the tuple again (D[1]=data,D[2]=labels)
#     X = D[1]
#     Y = D[2]
#     for leaf in Tree.leaves
#         indices = get_data_indices(get(leaf.Θ),X)  #idea: store the indices in the node
#         if length(indices)>0   #can get rid of this if!
#             y = Y[indices]   #get labels of present data points
#             for k in 1:length(leaf.c)
#                 leaf.c[k] = length(y[y.==k])  #update how many datapoints are at that leaf of each class
#                 leaf.tab[k] = min(leaf.c[k],1) #update: class present or not
#             end
#         end
#     end
# end

# # uses the leaf node counts to get the internal node counts
# function initialize_posterior_counts!(Tree,D)  #Tree=tree=MT, D=tuple 
#     initialize_posterior_leaf_counts!(Tree,D)
#     for leaf in Tree.leaves
#         j = leaf
#         while true
#             if j.node_type[2]==false
#                 for k in 1:length(j.c)
#                     j.c[k] = get(j.left).tab[k]+get(j.right).tab[k]
#                 end
#             end
#             for k in 1:length(j.c)
#                 j.tab[k] = min(j.c[k],1)
#             end
#             if j.node_type[3]==true
#                 break
#             else
#                 j = get(j.parent)
#             end
#         end
#     end
# end

# # gamma is usually set to 10*dimensionality (we set that as default parameter)
# # which is done above somewhere
# function compute_predictive_posterior_distribution!(Tree, γ=1) #Tree=tree=MT
#     J = [get(Tree.root)] #intialise array and put the root in
#     while (size(J,1) != 0) 
#         j = shift!(J)  #gets the top element and removes it from the array(like pop! in other languages...)
#         if (j.node_type[3]==true) #is it a root?
#             p = ones(length(j.c))./length(j.c) #uniform distribution over the length of j.c (#classes at j)
#             d = exp(-γ*(j.τ)) #just an exponential function, no dist! 
#             else  #not a root
#             d = exp(-γ*(j.τ-get(j.parent).τ))
#             p = get(j.parent).Gₚ
#         end
#         for k in 1:length(j.c) # loop over classes 
#             j.Gₚ[k] = (1/(sum(j.c)))*(j.c[k]-d*j.tab[k]+sum(j.tab)*p[k])
#         end
#         if sum(j.node_type)==0  #this should almost certainly be removed!
#             break
#         end
#         if j.node_type[2] == false
#             push!(J, get(j.left))
#             push!(J, get(j.right))
#         end
#     end
# end

# # the algorithm requires computing an expectation
# # empirically
# function expected_discount(nⱼ, Δⱼ,γ=1)
#     Δ = rand(Truncated(Exponential(1/nⱼ),0,Δⱼ),10000)
#     return mean(exp.(-γ*Δ))
# end

# # predict te class probs
# function predict!(T,x, γ=1) #T=Tree=tree=MT, x is a single datapoint
#     j = get(T.root)
#     not_sep = 1
#     s = zeros(size(j.c,1))  #s is the prediciton probability we return for each class 
#     while true
#         if (sum(j.node_type)==0) # this can be removed (most definitly)
#             break
#         end
#         if (j.node_type[3] == true)
#             Δⱼ = j.τ
#         else
#             Δⱼ = j.τ - get(j.parent).τ
#         end
#         nⱼ=0
#         for d in size(x,1) #loop over dimensions
#             nⱼ += max(x[d]-get(j.Θ).Intervals[d,2],0) + max(get(j.Θ).Intervals[d,1]-x[d],0)
#         end
#         pⱼ = 1-exp(Δⱼ*nⱼ)  #this is a probability!, just the exponential function no sampling 
#         # yes this part does add nodes to the tree!
#         # although i've never seen it called...
#         if pⱼ > 0 
#             # x branches
#             jₓ = Mondrian_Node()
#             if (j == get(j.parent).left)
#                 get(j.parent).left = jₓ
#             else
#                 get(j.parent).right = jₓ
#             end
#             jₓ.parent = get(j.parent)
#             j.parent = jₓ
#             jₓ.left = j
#             jₓ.right = Mondrian_Node()
#             d = expected_discount(nⱼ, Δⱼ, γ)
#             for k in 1:length(j.c)
#                 jₓ.c[k] = min(j.c[k],1)
#             end
#             jₓ.tab = jₓ.c
#             for k in 1:length(jₓ.c)
#                 jₓ.Gₚ[k] = 1/(sum(jₓ.c)+1)*(jₓ.c[k] - d*jₓ.tab[k]+d*sum(jₓ.tab)*get(jₓ.parent).Gₚ[k])
#             end
#             for k in 1:length(s)
#                 s[k] += not_sep*(1-pⱼ)*jₓ.Gₚ[k]
#             end
#         end
#         if j.node_type[2] == true
#             for k in 1:length(s)
#                 s[k] += not_sep*(1-pⱼ)*j.Gₚ[k]
#             end
#             return s
#         else
#             not_sep = not_sep*(1-pⱼ)
#             if x[get(j.δ)] <= get(j.ζ)
#                 j = get(j.left)
#             else
#                 j = get(j.right)
#             end
#         end
#     end
# end

# # compute depth
# function depth(Tree::Mondrian_Tree)
#     j = Tree.leaves[1]
#     d = 0
#     while !(j.node_type[3])
#         j = get(j.parent)
#         d+=1
#     end
#     return d
# end

depth (generic function with 1 method)

In [3]:
# # utils that should come from a file

# function get_count(j::Mondrian_Node, Y::Array{Int64}, class_num::Int64)
#     j.tab = zeros(class_num)
#     j.c = zeros(class_num)
#     for i in 1:class_num
#         j.c[i] = length(Y[Y.==i])
#         j.tab[i] = min(j.c[i],1)
#     end
# end

# function Sample_Mondrian_Tree!(Tree::Mondrian_Tree,
#                                λ::Float64,
#                                X::Array{Float64,2},
#                                Y::Array{Int64})
#     # initialise the tree
#     e = Mondrian_Node(0.0,[false,false,true])
#     Tree.root = e
#     Θ = Axis_Aligned_Box(get_intervals(X))
#     e.Θ = Θ
#     get_count(e,Y, length(unique(Y)))
#     e.Gₚ = zeros(size(unique(Y),1))
#     Sample_Mondrian_Block!(e, Θ, λ, Tree, X, Y)
#     return Tree
# end

# function Sample_Mondrian_Block!(j::Mondrian_Node,
#                                 Θ::Axis_Aligned_Box,
#                                 λ::Float64,
#                                 Tree::Mondrian_Tree,
#                                 X::Array{Float64,2},
#                                 Y::Array{Int64})
#     # sample the time
#     E = rand(Exponential(1/Linear_dimension(Θ)))
#     if j.node_type[3]==true
#         τₚ = 0
#     else
#         τₚ = (get(j.parent)).τ
#     end
#     # if split occured in time
#     if τₚ + E < λ
#         # get split dimension and cut position
#         # A2 -> lines 6,7
#         d,x = sample_split_dimension(Θ)
#         # update node j's data
#         j.δ = d
#         j.ζ = x
#         j.τ = τₚ+E
#         Θᴸ = copy(Θ)
#         # look at this copy
#         Θᴿ = copy(Θ)
#         # Left and Right children have constricted boxes
#         Θᴸ.Intervals[d,2]=x
#         Θᴿ.Intervals[d,1]=x
#         # check there is actually data here
#         Xᴿ = get_data_indices(Θᴿ,X,d)
#         Xᴸ = get_data_indices(Θᴸ,X,d)
#         # strictly binary tree
#         if (size(Xᴿ,1)>0 && size(Xᴸ,1)>0)
#             right = Mondrian_Node(0.0, [true,false,false])
#             right.parent = j
#             # data changes A2 -> lines 8,9,10
#             right.Θ = Θᴿ
#             get_count(right, Y[Xᴿ], length(j.c))
#             right.Gₚ=zeros(size(j.c,1))
#             j.right = right

#             left = Mondrian_Node(0.0, [true,false,false])
#             left.parent = j
#             left.Θ = Θᴸ
#             get_count(left, Y[Xᴸ], length(j.c))
#             left.Gₚ = zeros(size(j.c,1))
#             j.left = left

#             # recurse
#             Sample_Mondrian_Block!(left, get(left.Θ), λ, Tree, X[Xᴸ,:], Y[Xᴸ])
#             Sample_Mondrian_Block!(right, get(right.Θ),λ, Tree, X[Xᴿ,:], Y[Xᴿ])
#         # set j as a leaf for no data/ not binary
#         else
#             j.τ = λ
#             j.node_type = [false,true,false]
#             push!(Tree.leaves,j)
#             return
#         end
#     # set j as leaf for time out
#     else
#         j.τ = λ
#         j.node_type = [false,true,false]
#         push!(Tree.leaves,j)
#         return
#     end
# end

# # only check indices against the changed dimension CF lines 93-97
# function get_data_indices(Θ::Axis_Aligned_Box, X::Array{Float64,2}, dim::Int64)
#     # this function cause large memory allocation according
#     # to @time but the system does not record any
#     # large memory allocation -> ram does not get increased
#     # at all!
#     indices = []
#     for i in 1:size(X,1)
#         if !(X[i,dim] < Θ.Intervals[dim,1] || X[i,dim] > Θ.Intervals[dim,2])
#             push!(indices,i)
#         end
#     end
#     return indices
# end
# # returns any data from D contained in the boxes of Θ
# function get_data_indices(Θ::Axis_Aligned_Box, X::Array{Float64,2})
#     # this function cause large memory allocation according
#     # to @time but the system does not record any
#     # large memory allocation -> ram does not get increased
#     # at all!
#     indices = []
#     include = false
#     for i in 1:size(X,1)
#         for j in 1:size(Θ.Intervals,1)
#             if (X[i,j] < Θ.Intervals[j,1] || X[i,j] > Θ.Intervals[j,2])
#                 include = false
#                 break
#             end
#             include = true
#         end
#         if (include)
#             push!(indices, i)
#         end
#     end
#     return indices
# end

# # gamma is usually set to 10*dimensionality
# # which is done above somewhere
# function compute_predictive_posterior_distribution!(Tree::Mondrian_Tree,
#                                                     γ::Real)
#     J = [get(Tree.root)]
#     while (size(J,1) != 0)
#         j = shift!(J)
#         if (j.node_type[3]==true)
#             p = ones(length(j.c))./length(j.c)
#             d = exp(-γ*(j.τ))
#         else
#             d = exp(-γ*(j.τ-get(j.parent).τ))
#             p = get(j.parent).Gₚ
#         end
#         for k in 1:length(j.c)
#             j.Gₚ[k] = (1/(sum(j.c)))*(j.c[k]-d*j.tab[k]+sum(j.tab)*p[k])
#         end
#         if sum(j.node_type)==0
#             break
#         end
#         if j.node_type[2] == false
#             push!(J, get(j.left))
#             push!(J, get(j.right))
#         end
#     end
# end

compute_predictive_posterior_distribution! (generic function with 3 methods)

#### START OF MY WORK

In [2]:
#Go at implementing Alg 3

function Extend_Mondrian_Tree!(T,λ,D) #Mondrian Tree, λ, D -> new training instance as a tuple
    ϵ=get(T.root)
    Extend_Mondrian_Block!(T,λ,ϵ,D) #Input: Mondrian Tree, lifetime, rootnode, new training instance
    return T
end

Extend_Mondrian_Tree! (generic function with 1 method)

In [3]:
# Go at implementing Alg 4

function Extend_Mondrian_Block!(T,λ,j,D) # Input: Mondrian Tree, lifetime, node, new training instance as a tuple
    E = rand(Exponential(1/Extended_dimension(get(j.Θ),D[1])))
    if j.node_type[3]==true #if its a root
        τₚ = 0
    else
        τₚ = (get(j.parent)).τ
    end
    # if split occured in time
    if τₚ + E < j.τ
        d,x= sample_extended_split_dimension(get(j.Θ),D[1])
        Θ = update_intervals(get(j.Θ),D)
        if j.node_type[3]==true                                             ## check if we replace the root
            j_wave=Mondrian_Node(E,[false,false,true],d,x,Θ,zeros(size(get(j.tab))) ,zeros(size(get(j.c))),zeros(size(get(j.Gₚ))))
            j.node_type=[true,false,false]
        else
            j_wave=Mondrian_Node(parent=get(j.parent),get(j.parent).τ+E,[true,false,false],d,x,Θ,get(j.parent).tab,get(j.parent).c,get(j.parent).Gₚ)
            if j == get(j.parent).left
                get(j.parent).left = j_wave
            else
                get(j.parent).right = j_wave
            end
        end
            
        j.parent=j_wave
        j_prime = Mondrian_Node(0.0, [true,false,false])   #initialise new sibling to j
        j_prime.parent = j_wave 
        j_prime.tab = zeros(size(j_wave.tab)) 
        j_prime.c = zeros(size(j_wave.tab))   
        j_prime.Gₚ=zeros(size(j_wave.c,1))
            
        if D[1][d] > x    
            j_wave.left = j
            j_wave.right = j_prime

                if get(j.Θ.Intervals[2][d])> ζ    #adapt box of j
                    j.Θ.Intervals[2][d] = ζ
                end
            j_prime.Θ = j_wave.Θ
            j_wave.Θ.Intervals[1][d] = ζ
            Sample_Mondrian_Block!(j_prime,get(j_prime.Θ),λ,T,D[1],D[2])
        else
            j_wave.left = j_prime
            j_wave.right = j
            
            if get(j.Θ.Intervals[1][d])< ζ 
                j.Θ.Intervals[1][d] = ζ
            end
            j_prime.Θ = j_wave.Θ
            j_wave.Θ.Intervals[2][d] = ζ
            Sample_Mondrian_Block!(j_prime,get(j_prime.Θ),λ,T,D[1],D[2])
        end
                    
    
    else
        Θ = update_intervals(get(j.Θ),D)
        j.Θ=Θ
        if j.node_type != [false,true,false]
            if D[1][get(j.δ)] < get(j.ζ)
                Extend_Mondrian_Block!(T,λ,get(j.left),D)
            else
                Extend_Mondrian_Block!(T,λ,get(j.right),D)
            end
        end
    end       
end

Extend_Mondrian_Block! (generic function with 1 method)

In [4]:
#utils for Alg 3 and 4 (need to go in a file)

# ∑ᵢ₌₁ᴰmax(lⱼ-x,0)+max(x-uⱼ,0)
function Extended_dimension(Θ::Axis_Aligned_Box,datapoint) #input: boxes, datapoint (no label)
    s = 0
    for i in 1:Θ.D
        s += max(Θ.Intervals[i,1]-datapoint[i],0)+max(datapoint[i]-Θ.Intervals[i,2],0)        
    end
    return s
end

function sample_extended_split_dimension(Θ,datapoint) #input: boxes, datapoint (no label)
    p_k = zeros(Θ.D)
    for i in 1:Θ.D
        p_k[i] = max(Θ.Intervals[i,1]-datapoint[i],0)+max(datapoint[i]-Θ.Intervals[i,2],0)
    end
    p_k = p_k ./ Extended_dimension(Θ,datapoint)  #create the probability array
    d = rand(Categorical(p_k))        # sample from it
    
    if datapoint[d] > Θ.Intervals[d,2]
    x = rand(Uniform(Θ.Intervals[d,2][1],datapoint[d]))#sample the split loc. uniformly from that interval
    else
    x = rand(Uniform(datapoint[d],Θ.Intervals[d,1][1]))
    end
    return d,x
end

function Mondrian_Node(parent,τ,node_type,δ,ζ,Θ,tab, c, Gₚ)  #constructor used in Alg4 (Extended_Mond)
    N = Mondrian_Node(parent,
                      left::Nullable{Mondrian_Node},
                      right::Nullable{Mondrian_Node},
                      τ,node_type,
                      δ,
                      ζ,
                      Θ,
                      tab,
                      c,
                      Gₚ,
                      Array{Int}())
    return N
end


function Mondrian_Node(τ,node_type,δ,ζ,Θ,tab,c,Gₚ)  #constructor used in Alg4 
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      left::Nullable{Mondrian_Node},
                      right::Nullable{Mondrian_Node},
                      left,
                      right,
                      τ,node_type,
                      δ,
                      ζ,
                      Θ,
                      tab,
                      c,
                      Gₚ,
                      Array{Int}())
    return N
end

function update_intervals(Θ::Axis_Aligned_Box,D) #take θ and Datapoint as a tuple
    Intervals = Θ.Intervals
   for i=1:length(Θ.Intervals[:,1])
       l=min(Intervals[i,1],D[1][i])
       u=max(Intervals[i,2],D[1][i])
       Intervals[i,:]=[l,u]
    end 
    Θ = Axis_Aligned_Box(Intervals,Θ.D)
    return Θ
end

update_intervals (generic function with 1 method)

### TEST 

In [5]:
using MLBase
function Fakedata(n,dim) 
    x = randn(n,dim)
    y = (sum(exp.(x)/(1+exp.(x)),2)).>0.5
    return 1.0*x,1*y[:,1]
end

d=5
X, Y = Fakedata(1000,d);
Y = Y.+1;

x,y = Fakedata(1,d)
y = y.+1;

In [6]:
D=[X,Y]
d_new = [x,y] 
T = Mondrian_Tree()

Mondrian_Tree(Nullable{Mondrian_Node}(), Mondrian_Node[])

In [7]:
T_classic = Sample_Mondrian_Tree!(T,1e9,D[1],D[2]);  #compute base tree

In [2]:
T_online = Extend_Mondrian_Tree!(T_classic,1e9,d_new);  #extent base tree

### Problem: The Mondrian_Tree function does not return, tree can therefore not be used for extension!!

LoadError: [91mUndefVarError: Extend_Mondrian_Tree! not defined[39m

In [39]:
# create new dataset including previous training data + new datapoint
All_data = zeros(1001,5)
All_data[1:1000,1:5]=X[1:1000,1:5]
All_data[1001,1:5]=x[1,1:5]

all_labels = Array{Int64}(1001,1)
all_labels[1:1000,:]=Y
all_labels[1001,:]=y[1,1]

1

In [44]:
T_full = Sample_Mondrian_Tree!(T,1e9,All_data,all_labels);  #compute full tree of the whole dataset

### Compare prediction accuracy

In [51]:
compute_predictive_posterior_distribution!(T_full,10*size(All_data,2))

In [52]:
pred = []
    for i in 1:size(All_data,1)
        p = predict!(T_full,All_data[i,:],10*size(All_data,2))
        if p[1] > p[2]
            push!(pred,1)
        else
            push!(pred,2)
        end
    end


In [53]:
counter = 0
for i=1:1001
    if pred[i]==all_labels[i]
        counter = counter+1
    end
end
counter = counter /1001

0.7112887112887113

In [54]:
compute_predictive_posterior_distribution!(T_online,10*size(All_data,2))

In [55]:
 pred = []
        for i in 1:size(All_data,1)
            p = predict!(T_online,All_data[i,:],10*size(All_data,2))
            if p[1] > p[2]
                push!(pred,1)
            else
                push!(pred,2)
            end
        end

In [56]:
counter = 0
for i=1:1001
    if pred[i]==all_labels[i]
        counter = counter+1
    end
end
counter = counter /1001

0.7112887112887113

## Conclusion: get the same accuracy weather we incorperate the datapoint later or immediatly

## To Do:

Clean up utils files, test on more classes, test on extending to more datapoints successively, implement paused mondrians 